## Python porting of Support Resistance Channels TradingView Indicator  
by LonesomeTheBlue

<https://www.tradingview.com/script/Ej53t8Wv-Support-Resistance-Channels/>

>Developed by [@edyatl](https://github.com/edyatl) May 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
// This source code is subject to the terms of the Mozilla Public License 2.0 at https://mozilla.org/MPL/2.0/
// © LonesomeTheBlue
 
//@version=4
study("Support Resistance Channels", "SRchannel", overlay = true, max_bars_back = 501)
prd = input(defval = 10, title="Pivot Period", minval = 4, maxval = 30, group = "Settings 🔨", tooltip="Used while calculating Pivot Points, checks left&right bars")
ppsrc = input(defval = 'High/Low', title="Source", options = ['High/Low', 'Close/Open'], group = "Settings 🔨", tooltip="Source for Pivot Points")
ChannelW = input(defval = 5, title = "Maximum Channel Width %", minval = 1, maxval = 8, group = "Settings 🔨", tooltip="Calculated using Highest/Lowest levels in 300 bars")
minstrength = input(defval = 1, title = "Minimum Strength", minval = 1, group = "Settings 🔨", tooltip = "Channel must contain at least 2 Pivot Points")
maxnumsr = input(defval = 6, title = "Maximum Number of S/R", minval = 1, maxval = 10, group = "Settings 🔨", tooltip = "Maximum number of Support/Resistance Channels to Show") - 1
loopback = input(defval = 290, title = "Loopback Period", minval = 100, maxval = 400, group = "Settings 🔨", tooltip="While calculating S/R levels it checks Pivots in Loopback Period")
res_col = input(defval = color.new(color.red, 75), title = "Resistance Color", group = "Colors 🟡🟢🟣")
sup_col = input(defval = color.new(color.lime, 75), title = "Support Color", group = "Colors 🟡🟢🟣")
inch_col = input(defval = color.new(color.gray, 75), title = "Color When Price in Channel", group = "Colors 🟡🟢🟣")
showpp = input(defval = false, title = "Show Pivot Points", group = "Extras ⏶⏷")
showsrbroken = input(defval = false, title = "Show Broken Support/Resistance", group = "Extras ⏶⏷")
showthema1en = input(defval = false, title = "MA 1", inline = "ma1")
showthema1len = input(defval = 50, title = "", inline = "ma1")
showthema1type = input(defval = "SMA", title = "", options = ["SMA", "EMA"], inline = "ma1")
showthema2en = input(defval = false, title = "MA 2", inline = "ma2")
showthema2len = input(defval = 200, title = "", inline = "ma2")
showthema2type = input(defval = "SMA", title = "", options = ["SMA", "EMA"], inline = "ma2")

ma1 = showthema1en ? (showthema1type == "SMA" ? sma(close, showthema1len) : ema(close, showthema1len)) : na
ma2 = showthema2en ? (showthema2type == "SMA" ? sma(close, showthema2len) : ema(close, showthema2len)) : na

plot(ma1, color = not na(ma1) ? color.blue : na)
plot(ma2, color = not na(ma2) ? color.red : na)

// get Pivot High/low
float src1 =  ppsrc == 'High/Low' ? high : max(close, open)
float src2 =  ppsrc == 'High/Low' ? low: min(close, open)
float ph = pivothigh(src1, prd, prd)
float pl = pivotlow(src2, prd, prd) 

// draw Pivot points
plotshape(ph and showpp, text = "H",  style = shape.labeldown, color = na, textcolor = color.red, location = location.abovebar, offset = -prd)
plotshape(pl and showpp, text = "L",  style = shape.labelup, color = na, textcolor = color.lime, location = location.belowbar, offset = -prd)

//calculate maximum S/R channel width
prdhighest =  highest(300)
prdlowest = lowest(300)
cwidth = (prdhighest - prdlowest) * ChannelW / 100

// get/keep Pivot levels
var pivotvals= array.new_float(0)
var pivotlocs= array.new_float(0)
if ph or pl
    array.unshift(pivotvals, ph ? ph : pl)
    array.unshift(pivotlocs, bar_index)
    for x = array.size(pivotvals) - 1 to 0
        if bar_index - array.get(pivotlocs, x) > loopback // remove old pivot points
            array.pop(pivotvals)
            array.pop(pivotlocs)
            continue
        break

//find/create SR channel of a pivot point
get_sr_vals(ind)=>
    float lo = array.get(pivotvals, ind)
    float hi = lo
    int numpp = 0
    for y = 0 to array.size(pivotvals) - 1
        float cpp = array.get(pivotvals, y)
        float wdth = cpp <= hi ? hi - cpp : cpp - lo
        if wdth <= cwidth // fits the max channel width?
            if cpp <= hi
                lo := min(lo, cpp)
            else
                hi := max(hi, cpp)
                
            numpp := numpp + 20 // each pivot point added as 20
    [hi, lo, numpp] 

// keep old SR channels and calculate/sort new channels if we met new pivot point
var suportresistance = array.new_float(20, 0) // min/max levels
changeit(x, y)=>
    tmp = array.get(suportresistance, y * 2)
    array.set(suportresistance, y * 2, array.get(suportresistance, x * 2))
    array.set(suportresistance, x * 2, tmp)
    tmp := array.get(suportresistance, y * 2 + 1)
    array.set(suportresistance, y * 2 + 1, array.get(suportresistance, x * 2 + 1))
    array.set(suportresistance, x * 2 + 1, tmp)
    
if ph or pl
    supres = array.new_float(0)  // number of pivot, strength, min/max levels
    stren = array.new_float(10, 0)
    // get levels and strengs
    for x = 0 to array.size(pivotvals) - 1
        [hi, lo, strength] = get_sr_vals(x)
        array.push(supres, strength)
        array.push(supres, hi)
        array.push(supres, lo)
    
    // add each HL to strengh
    for x = 0 to array.size(pivotvals) - 1
        h = array.get(supres, x * 3 + 1)
        l = array.get(supres, x * 3 + 2)
        s = 0
        for y = 0 to loopback
            if (high[y] <= h and high[y] >= l) or
               (low[y] <= h and low[y] >= l)
                s := s + 1
        array.set(supres, x * 3, array.get(supres, x * 3) + s)
    
    //reset SR levels
    array.fill(suportresistance, 0)
    // get strongest SRs
    src = 0
    for x = 0 to array.size(pivotvals) - 1
        stv = -1. // value
        stl = -1 // location
        for y = 0 to array.size(pivotvals) - 1
            if array.get(supres, y * 3) > stv and array.get(supres, y * 3) >= minstrength * 20
                stv := array.get(supres, y * 3)
                stl := y
        if stl >= 0
            //get sr level
            hh = array.get(supres, stl * 3 + 1)
            ll = array.get(supres, stl * 3 + 2)
            array.set(suportresistance, src * 2, hh)
            array.set(suportresistance, src * 2 + 1, ll)
            array.set(stren, src, array.get(supres, stl * 3))
            
            // make included pivot points' strength zero 
            for y = 0 to array.size(pivotvals) - 1
                if (array.get(supres, y * 3 + 1) <= hh and array.get(supres, y * 3 + 1) >= ll) or
                   (array.get(supres, y * 3 + 2) <= hh and array.get(supres, y * 3 + 2) >= ll)
                    array.set(supres, y * 3, -1)

            src += 1
            if src >= 10
                break
    
    for x = 0 to 8
        for y = x + 1 to 9
            if array.get(stren, y) > array.get(stren, x)
                tmp = array.get(stren, y) 
                array.set(stren, y, array.get(stren, x))
                changeit(x, y)
                
    
get_level(ind)=>
    float ret = na
    if ind < array.size(suportresistance)
        if array.get(suportresistance, ind) != 0
            ret := array.get(suportresistance, ind)
    ret
    
get_color(ind)=>
    color ret = na
    if ind < array.size(suportresistance)
        if array.get(suportresistance, ind) != 0
            ret := array.get(suportresistance, ind) > close and array.get(suportresistance, ind + 1) > close ? res_col :
                   array.get(suportresistance, ind) < close and array.get(suportresistance, ind + 1) < close ? sup_col :
                   inch_col
    ret

var srchannels = array.new_box(10)
for x = 0 to min(9, maxnumsr)
    box.delete(array.get(srchannels, x))
    srcol = get_color(x * 2)
    if not na(srcol)
        array.set(srchannels, x, 
                  box.new(left = bar_index, top = get_level(x * 2), right = bar_index + 1, bottom = get_level(x * 2 + 1), 
                          border_color = srcol, 
                          border_width = 1,
                          extend = extend.both, 
                          bgcolor = srcol))

resistancebroken = false
supportbroken = false

// check if it's not in a channel
not_in_a_channel = true
for x = 0 to min(9, maxnumsr)
    if close <= array.get(suportresistance, x * 2) and close >= array.get(suportresistance, x * 2 + 1) 
        not_in_a_channel := false

// if price is not in a channel then check broken ones
if not_in_a_channel
    for x = 0 to min(9, maxnumsr)
        if close[1] <= array.get(suportresistance, x * 2) and close > array.get(suportresistance, x * 2)
            resistancebroken := true
        if close[1] >= array.get(suportresistance, x * 2 + 1) and close < array.get(suportresistance, x * 2 + 1)
            supportbroken := true

alertcondition(resistancebroken, title = "Resistance Broken", message = "Resistance Broken")
alertcondition(supportbroken, title = "Support Broken", message = "Support Broken")
plotshape(showsrbroken and resistancebroken, style = shape.triangleup, location = location.belowbar, color = color.new(color.lime, 0), size = size.tiny)
plotshape(showsrbroken and supportbroken, style = shape.triangledown, location = location.abovebar, color = color.new(color.red, 0), size = size.tiny)
```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

# Nicest style for plots
# sns.set(style="ticks")

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="ATOMUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-05-23 19:45:00,10.49300000,10.49700000,10.48100000,10.48300000,4755.84000000,2023-05-23 19:59:59.999,49896.15671000,165,1559.31000000,16362.08305000,0
496,2023-05-23 20:00:00,10.48400000,10.49600000,10.47400000,10.48600000,7379.44000000,2023-05-23 20:14:59.999,77363.36672000,211,2627.28000000,27545.04543000,0
497,2023-05-23 20:15:00,10.48600000,10.49300000,10.48100000,10.49300000,3275.89000000,2023-05-23 20:29:59.999,34353.56444000,171,2386.50000000,25026.74403000,0
498,2023-05-23 20:30:00,10.49300000,10.50200000,10.49100000,10.50000000,2899.60000000,2023-05-23 20:44:59.999,30434.58858000,153,1567.98000000,16457.52579000,0
499,2023-05-23 20:45:00,10.50000000,10.51700000,10.49900000,10.50900000,3668.62000000,2023-05-23 20:59:59.999,38561.99885000,157,2454.15000000,25798.19124000,0


<IPython.core.display.Javascript object>

### Inputs

In [8]:
# "Pivot Period", min = 4, max = 30, "Used while calculating Pivot Points, checks left&right bars"
prd: int = 10

# "Source", ['High/Low', 'Close/Open'], "Source for Pivot Points"
ppsrc: str = "High/Low"

# "Maximum Channel Width %", min = 1, max = 8, "Calculated using Highest/Lowest levels in 300 bars"
ChannelW: int = 5

# "Minimum Strength", minval = 1, "Channel must contain at least 2 Pivot Points"
minstrength: int = 1

# "Maximum Number of S/R", min = 1, max = 10, "Maximum number of Support/Resistance Channels to Show" - 1
maxnumsr: int = 6 - 1

# "Loopback Period", min = 100, max = 400, "While calculating S/R levels it checks Pivots in Loopback Period"
loopback: int = 290

# res_col = input(defval = color.new(color.red, 75), title = "Resistance Color", group = "Colors 🟡🟢🟣")
# sup_col = input(defval = color.new(color.lime, 75), title = "Support Color", group = "Colors 🟡🟢🟣")
# inch_col = input(defval = color.new(color.gray, 75), title = "Color When Price in Channel", group = "Colors 🟡🟢🟣")

showpp: bool = False  # "Show Pivot Points"
showsrbroken: bool = False  # "Show Broken Support/Resistance"

showthema1en: bool = True  # "MA 1"
showthema1len: int = 50  # "ma1 length"
showthema1type: str = "SMA"  # ["SMA", "EMA"]

showthema2en: bool = True  # "MA 2"
showthema2len: int = 200  # "ma2 length"
showthema2type: str = "SMA"  # ["SMA", "EMA"]

<IPython.core.display.Javascript object>

In [9]:
close: np.ndarray = data["close"].to_numpy(dtype=np.double)
open: np.ndarray = data["open"].to_numpy(dtype=np.double)
high: np.ndarray = data["high"].to_numpy(dtype=np.double)
low: np.ndarray = data["low"].to_numpy(dtype=np.double)

<IPython.core.display.Javascript object>

In [10]:
# min/max levels
suportresistance: np.ndarray = np.zeros(20, dtype=np.double)

<IPython.core.display.Javascript object>

### Functions

In [11]:
# find/create SR channel of a pivot point
def get_sr_vals(ind: int) -> tuple:
    lo: np.double = pivotvals[ind]
    hi: np.double = lo
    numpp: int = 0

    for y in range(len(pivotvals)):
        cpp: np.double = pivotvals[y]
        wdth: np.double = hi - cpp if cpp <= hi else cpp - lo
        if wdth <= cwidth[bar_index]:  # fits the max channel width?
            if cpp <= hi:
                lo = min(lo, cpp)
            else:
                hi = max(hi, cpp)
            numpp += 20  # each pivot point added as 20
    return hi, lo, numpp

<IPython.core.display.Javascript object>

In [12]:
# for i in range(len(pivotvals)):
#     print(i, get_sr_vals(i))

<IPython.core.display.Javascript object>

In [13]:
# keep old SR channels and calculate/sort new channels if we met new pivot point
def changeit(x: int, y: int):
    tmp: np.double = suportresistance[y * 2]
    suportresistance[y * 2] = suportresistance[x * 2]
    suportresistance[x * 2] = tmp
    tmp = suportresistance[(y * 2) + 1]
    suportresistance[(y * 2) + 1] = suportresistance[(x * 2) + 1]
    suportresistance[(x * 2) + 1] = tmp

<IPython.core.display.Javascript object>

In [14]:
def get_level(ind: int) -> np.double:
    ret: np.double = None
    if ind < len(suportresistance):
        if suportresistance[ind] != 0:
            return suportresistance[ind]
    return ret

<IPython.core.display.Javascript object>

In [15]:
def get_color(ind: int, _close: np.double) -> str:
    ret: str = None
    if ind < len(suportresistance):
        if suportresistance[ind] != 0:
            ret = (
                "res_col"
                if suportresistance[ind] > _close and suportresistance[ind + 1] > _close
                else "sup_col"
                if suportresistance[ind] < _close and suportresistance[ind + 1] < _close
                else "inch_col"
            )
    return ret

<IPython.core.display.Javascript object>

In [16]:
def exclude_repeats(pv: np.ndarray, smpl: np.ndarray, sp: int) -> np.ndarray:
    """Exclude repeating values"""
    for i in range(sp, len(pv) - sp):
        for j in range(sp):
            if pv[i] == smpl[i + 1 : i + 1 + sp][j]:
                pv[i] = np.NaN
            if pv[i] == smpl[i - sp : i][j]:
                pv[i] = np.NaN
    return pv

<IPython.core.display.Javascript object>

In [17]:
def pivothigh(high: np.ndarray, left: int, right: int) -> np.ndarray:
    pivots = np.roll(tl.MAX(high, left + 1 + right), -right)
    pivots[pivots != high] = np.NaN

    # Exclude repeating pivot highs
    exclude_repeats(pivots, high, right)
    return pivots

<IPython.core.display.Javascript object>

In [18]:
def pivotlow(low: np.ndarray, left: int, right: int) -> np.ndarray:
    pivots = np.roll(tl.MIN(low, left + 1 + right), -right)
    pivots[pivots != low] = np.NaN

    # Exclude repeating pivot lows
    exclude_repeats(pivots, low, right)
    return pivots

<IPython.core.display.Javascript object>

### Main

In [19]:
ma1 = (
    (
        tl.SMA(close, showthema1len)
        if showthema1type == "SMA"
        else tl.EMA(close, showthema1len)
    )
    if showthema1en
    else None
)

<IPython.core.display.Javascript object>

In [20]:
ma2 = (
    (
        tl.SMA(close, showthema2len)
        if showthema2type == "SMA"
        else tl.EMA(close, showthema2len)
    )
    if showthema2en
    else None
)

<IPython.core.display.Javascript object>

In [21]:
# get Pivot High/low
src1: np.ndarray = high if ppsrc == "High/Low" else np.maximum(close, open)
src2: np.ndarray = low if ppsrc == "High/Low" else np.minimum(close, open)

ph: np.ndarray = pivothigh(src1, prd, prd)
pl: np.ndarray = pivotlow(src2, prd, prd)

<IPython.core.display.Javascript object>

In [22]:
# calculate maximum S/R channel width
prdhighest: np.ndarray = tl.MAX(high, 300)
prdlowest: np.ndarray = tl.MIN(low, 300)
cwidth: np.ndarray = (prdhighest - prdlowest) * ChannelW / 100

<IPython.core.display.Javascript object>

In [23]:
# get/keep Pivot levels
pivotvals: np.ndarray = np.zeros(0, dtype=np.double)
pivotlocs: np.ndarray = np.zeros(0, dtype=np.int)

<IPython.core.display.Javascript object>

In [24]:
for bar_index, (_ph, _pl, ph_nan, pl_nan) in enumerate(
    zip(ph, pl, np.isnan(ph), np.isnan(pl))
):
    if not ph_nan or not pl_nan:
        pivotvals = np.insert(pivotvals, 0, [_ph if not ph_nan else _pl])
        pivotlocs = np.insert(pivotlocs, 0, [bar_index])

<IPython.core.display.Javascript object>

In [25]:
# for bar_index in reversed(range(len(high))):
for x in reversed(range(len(pivotvals))):
    bar_index = len(high) - 1
    # remove old pivot points
    if bar_index - pivotlocs[x] > loopback:
        pivotvals = np.delete(pivotvals, x, 0)
        pivotlocs = np.delete(pivotlocs, x, 0)

<IPython.core.display.Javascript object>

In [26]:
for bar_index, (_ph, _pl, ph_nan, pl_nan) in enumerate(
    zip(ph, pl, np.isnan(ph), np.isnan(pl))
):
    if not ph_nan or not pl_nan:
        # number of pivot, strength, min/max levels
        supres: np.ndarray = np.zeros(0, dtype=np.double)
        stren: np.ndarray = np.zeros(10, dtype=np.double)

        # get levels and strengs
        for x1 in range(len(pivotvals)):
            hi, lo, strength = get_sr_vals(x1)
            supres = np.append(supres, strength)
            supres = np.append(supres, hi)
            supres = np.append(supres, lo)

        # add each HL to strengh
        for x2 in range(len(pivotvals)):
            h: np.double = supres[x2 * 3 + 1]
            l: np.double = supres[x2 * 3 + 2]
            s: int = 0

            for y2 in range(loopback + 1):
                # TODO: rewrite for each HL after adding for loop
                if len(high[:bar_index]) - 1 < y2:
                    continue
                if (
                    high[:bar_index][-y2 - 1] <= h and high[:bar_index][-y2 - 1] >= l
                ) or (low[:bar_index][-y2 - 1] <= h and low[:bar_index][-y2 - 1] >= l):
                    s += 1
            supres[x2 * 3] = supres[x2 * 3] + s

        # reset SR levels
        suportresistance.fill(0)

        # get strongest SRs
        src: int = 0
        for x3 in range(len(pivotvals)):
            stv: np.double = -1.0  # value
            stl: int = -1  # location
            for y3 in range(len(pivotvals)):
                if supres[y3 * 3] > stv and supres[y3 * 3] >= minstrength * 20:
                    stv = supres[y3 * 3]
                    stl = y3
            if stl >= 0:
                # get sr level
                hh = supres[stl * 3 + 1]
                ll = supres[stl * 3 + 2]
                suportresistance[src * 2] = hh
                suportresistance[src * 2 + 1] = ll
                stren[src] = supres[stl * 3]

                # make included pivot points' strength zero
                for y32 in range(len(pivotvals)):
                    if (supres[y32 * 3 + 1] <= hh and supres[y32 * 3 + 1] >= ll) or (
                        supres[y32 * 3 + 2] <= hh and supres[y32 * 3 + 2] >= ll
                    ):
                        supres[y32 * 3] = -1

                src += 1
                if src >= 10:
                    break

        for x4 in range(9):
            for y4 in range(x4 + 1, 10):
                if stren[y4] > stren[x4]:
                    tmp = stren[y4]
                    stren[y4] = stren[x4]
                    changeit(x4, y4)

<IPython.core.display.Javascript object>

In [27]:
pivotvals

array([10.571, 10.54 , 10.45 , 10.623, 10.445, 10.526, 10.44 , 10.584,
       10.461, 10.325, 10.505, 10.403, 10.41 , 10.558, 10.429, 10.576,
       10.536, 10.619, 10.558, 10.546])

<IPython.core.display.Javascript object>

In [28]:
for x in range(5):
    print(get_level(x * 2))
stren

10.571
10.54
10.45
10.41
10.623


array([106., 102.,  78.,  50.,  44.,  23.,  21.,   0.,   0.,   0.])

<IPython.core.display.Javascript object>

In [29]:
top_level: np.ndarray = np.zeros(10, dtype=np.double)
bot_level: np.ndarray = np.zeros(10, dtype=np.double)

for bar_index, _close in enumerate(close):
    for x in range(min(10, (maxnumsr + 1))):
        top_level[x] = np.nan
        bot_level[x] = np.nan

        srcol = get_color(x * 2, _close)
        if srcol:
            top_level[x] = get_level(x * 2)
            bot_level[x] = get_level(x * 2 + 1)

top_level = np.where(top_level != 0, top_level, np.nan)
bot_level = np.where(bot_level != 0, bot_level, np.nan)

<IPython.core.display.Javascript object>

In [30]:
top_level

array([10.571, 10.54 , 10.45 , 10.41 , 10.623, 10.505,    nan,    nan,
          nan,    nan])

<IPython.core.display.Javascript object>

In [31]:
bot_level

array([10.558, 10.526, 10.44 , 10.403, 10.619, 10.505,    nan,    nan,
          nan,    nan])

<IPython.core.display.Javascript object>

In [32]:
sr_res = pd.DataFrame(
    {
        "top_level": top_level,
        "bot_level": bot_level,
    }
)
sr_res.dropna()

,top_level,bot_level
0,10.571,10.558
1,10.540,10.526
2,10.450,10.440
3,10.410,10.403
4,10.623,10.619
5,10.505,10.505


<IPython.core.display.Javascript object>

In [33]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "high": high,
        "low": low,
        "ph": ph,
        "pl": pl,
        "ma1": ma1,
        "ma2": ma2,
        "prdhighest": prdhighest,
        "prdlowest": prdlowest,
        "cwidth": cwidth,
    }
)
res.tail(-278).head(22)

,open_time,high,low,ph,pl,ma1,ma2,prdhighest,prdlowest,cwidth
278,2023-05-21 13:30:00,10.539,10.525,NaN,NaN,10.55132,10.569450,NaN,NaN,NaN
279,2023-05-21 13:45:00,10.542,10.531,NaN,NaN,10.54984,10.569430,NaN,NaN,NaN
280,2023-05-21 14:00:00,10.544,10.531,NaN,NaN,10.54852,10.569315,NaN,NaN,NaN
281,2023-05-21 14:15:00,10.550,10.541,NaN,NaN,10.54770,10.569215,NaN,NaN,NaN
282,2023-05-21 14:30:00,10.558,10.531,10.558,NaN,10.54660,10.568910,NaN,NaN,NaN
283,2023-05-21 14:45:00,10.540,10.508,NaN,NaN,10.54518,10.568445,NaN,NaN,NaN
284,2023-05-21 15:00:00,10.543,10.509,NaN,NaN,10.54396,10.568055,NaN,NaN,NaN
285,2023-05-21 15:15:00,10.529,10.511,NaN,NaN,10.54296,10.567775,NaN,NaN,NaN
286,2023-05-21 15:30:00,10.533,10.410,NaN,10.41,10.54008,10.567115,NaN,NaN,NaN
287,2023-05-21 15:45:00,10.479,10.421,NaN,NaN,10.53802,10.566645,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [34]:
res["ph"].loc[res["ph"].notnull()]

27     10.920
61     10.654
93     10.630
126    10.628
143    10.642
161    10.601
202    10.621
228    10.619
261    10.576
282    10.558
314    10.505
376    10.584
393    10.526
430    10.623
460    10.540
480    10.571
Name: ph, dtype: float64

<IPython.core.display.Javascript object>

In [35]:
res["pl"].loc[res["pl"].notnull()][::-1]

456    10.450
419    10.445
389    10.440
372    10.461
323    10.325
308    10.403
286    10.410
269    10.429
253    10.536
224    10.558
212    10.546
159    10.519
130    10.522
110    10.511
94     10.408
76     10.483
44     10.546
Name: pl, dtype: float64

<IPython.core.display.Javascript object>

In [ ]:
smpl = np.array(res["pl"].loc[res["pl"].notnull()][::-1])
# smpl = np.insert(smpl, 4, smpl[3])
smpl

In [ ]:
sp = 3
pv = np.roll(tl.MAX(smpl, sp + 1 + sp), -sp)
pv[pv != smpl] = np.NaN
pv

In [ ]:
for i in range(sp, len(pv) - sp):
    for j in range(sp):
        if pv[i] == smpl[i + 1 : i + 1 + sp][j]:
            pv[i] = np.NaN
        if pv[i] == smpl[i - sp : i][j]:
            pv[i] = np.NaN
pv

In [ ]:
smpl[3 + 1 : 3 + 1 + 3]

In [ ]:
for r in range(sp, len(pv) - sp):
    print(r)

In [ ]:
rpv = np.where(smpl[4 : 4 + 3] == pv[4])
rpv

In [ ]:
rsp = pd.DataFrame(
    {
        "smpl": smpl,
        "pv": pv,
    }
)
rsp